<a href="https://colab.research.google.com/github/KRiver28/TIL/blob/master/4_4_SGNS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.stem import LancasterStemmer
from tensorflow.keras.layers import Input, Embedding, Dense, Dot, Activation, Flatten
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle

nltk.download('punkt')
nltk.download('gutenberg')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# 영문 소설 18개를 읽어와서 전처리를 수행한다.
n = 18
stemmer = LancasterStemmer()
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['and', 'but', 'the', 'for', 'would', 'shall'])

sent_stem = []
files = nltk.corpus.gutenberg.fileids()
for i, text_id in enumerate(files[:n]):
    text = nltk.corpus.gutenberg.raw(text_id)
    sentences = nltk.sent_tokenize(text)

    # 각 단어에 Lancaster stemmer를 적용한다.
    for sentence in sentences:
        word_tok = nltk.word_tokenize(sentence)
        stem = [stemmer.stem(word) for word in word_tok if word not in stopwords if len(word) > 2]
        sent_stem.append(stem)
    print('{}: {} ----- processed.'.format(i+1, text_id))

print("\n총 문장 개수 =", len(sent_stem))
print(sent_stem[0])

1: austen-emma.txt ----- processed.
2: austen-persuasion.txt ----- processed.
3: austen-sense.txt ----- processed.
4: bible-kjv.txt ----- processed.
5: blake-poems.txt ----- processed.
6: bryant-stories.txt ----- processed.
7: burgess-busterbrown.txt ----- processed.
8: carroll-alice.txt ----- processed.
9: chesterton-ball.txt ----- processed.
10: chesterton-brown.txt ----- processed.
11: chesterton-thursday.txt ----- processed.
12: edgeworth-parents.txt ----- processed.
13: melville-moby_dick.txt ----- processed.
14: milton-paradise.txt ----- processed.
15: shakespeare-caesar.txt ----- processed.
16: shakespeare-hamlet.txt ----- processed.
17: shakespeare-macbeth.txt ----- processed.
18: whitman-leaves.txt ----- processed.

총 문장 개수 = 94434
['emm', 'jan', 'aust', '1816', 'volum', 'chapt', 'emm', 'woodh', 'handsom', 'clev', 'rich', 'comfort', 'hom', 'happy', 'disposit', 'seem', 'unit', 'best', 'bless', 'ex', 'liv', 'near', 'twenty-one', 'year', 'world', 'littl', 'distress', 'vex']


In [3]:
# 단어사전
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sent_stem)

# 단어사전
word2idx = tokenizer.word_index
word2idx['<PAD>'] = 0
idx2word = {v:k for k, v in word2idx.items()}

print("사전 크기 =", len(word2idx))

사전 크기 = 32545


In [4]:
# 문장을 단어의 인덱스로 표현
sent_idx = tokenizer.texts_to_sequences(sent_stem)
# sent_idx[0]

# 학습 데이터 생성
# trigram
x_train_t = []   # target word
x_train_c = []   # context word

# positive data
for sent in sent_idx:
    if len(sent) < 3:
        continue

    for a, b, c in nltk.trigrams(sent):
        x_train_t.append(b)
        x_train_t.append(b)
        x_train_c.append(a)
        x_train_c.append(c)
y_train_pos = [1] * len(x_train_t)

In [5]:
# negative sampling. random이 오래 걸려서 아래처럼 일괄처리함.
ns_k = 1
x_train_nt = []   # negative target word
x_train_nc = []   # negative context word
for k in range(ns_k):
    r = np.random.choice(range(1, len(word2idx)), len(x_train_t))
    x_train_nt.extend(x_train_t.copy())
    x_train_nc.extend(list(r).copy())
y_train_neg = [0] * len(x_train_t) * ns_k

In [6]:
# positive + negative
x_target = x_train_t + x_train_nt
x_context = x_train_c + x_train_nc
y_train = y_train_pos + y_train_neg   

# shuffling
x_target, x_context, y_train = shuffle(x_target, x_context, y_train)

# list --> array 변환
x_target = np.array(x_target).reshape(-1, 1)
x_context = np.array(x_context).reshape(-1, 1)
y_train = np.array(y_train).reshape(-1, 1)

x_target.shape, x_context.shape, y_train.shape

((3607280, 1), (3607280, 1), (3607280, 1))

In [7]:
VOC_SIZE = len(word2idx)
EMB_SIZE = 32

x_input_t = Input(batch_shape=(None, 1))
x_input_c = Input(batch_shape=(None, 1))

SharedEmb = Embedding(VOC_SIZE, EMB_SIZE, name='emb')
x_emb_t = SharedEmb(x_input_t)
x_emb_c = SharedEmb(x_input_c)

y_output = Dot(axes=(2,2))([x_emb_t, x_emb_c])
y_output = Activation('sigmoid')(y_output)

model = Model([x_input_t, x_input_c], y_output)
model.compile(loss = 'binary_crossentropy', optimizer='adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 emb (Embedding)                (None, 1, 32)        1041440     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dot (Dot)                      (None, 1, 1)         0           ['emb[0][0]',                

In [8]:
# word --> word2vec을 확인하기 위한 모델
model_vec = Model(x_input_t, x_emb_t)

hist = model.fit([x_target, x_context], y_train, batch_size=20480, epochs=50)

def get_word2vec(word):
    stem_word = stemmer.stem(word)
    if stem_word not in word2idx:
        print('{}가 없습니다.'.format(word))
        return
    
    word2vec = model_vec.predict(np.array(word2idx[stem_word]).reshape(1,1))[0]
    return word2vec

Epoch 1/50
177/177 [==============================] - 6s 30ms/step - loss: 0.6500
Epoch 2/50
177/177 [==============================] - 5s 31ms/step - loss: 0.4452
Epoch 3/50
177/177 [==============================] - 5s 31ms/step - loss: 0.3891
Epoch 4/50
177/177 [==============================] - 5s 31ms/step - loss: 0.3778
Epoch 5/50
177/177 [==============================] - 5s 30ms/step - loss: 0.3720
Epoch 6/50
177/177 [==============================] - 5s 30ms/step - loss: 0.3674
Epoch 7/50
177/177 [==============================] - 5s 30ms/step - loss: 0.3633
Epoch 8/50
177/177 [==============================] - 5s 30ms/step - loss: 0.3595
Epoch 9/50
177/177 [==============================] - 5s 30ms/step - loss: 0.3561
Epoch 10/50
177/177 [==============================] - 5s 30ms/step - loss: 0.3530
Epoch 11/50
177/177 [==============================] - 6s 31ms/step - loss: 0.3501
Epoch 12/50
177/177 [==============================] - 5s 30ms/step - loss: 0.3470
Epoch 13/50
1

In [9]:
father = get_word2vec('father')
mother = get_word2vec('mother')
doctor = get_word2vec('doctor')

cosine_similarity(father, mother)

cosine_similarity(father, doctor)

father

array([[ 0.5190904 ,  0.6715766 ,  0.0031392 , -0.28505743,  1.8104464 ,
        -0.56192017,  1.1648088 , -1.0087848 ,  1.0738708 , -0.350968  ,
        -0.1595304 , -1.1930829 ,  1.3331943 ,  0.9594065 ,  0.53037125,
         0.16261335, -0.99865437, -0.51732546,  0.9014974 , -0.34467694,
         0.27750683,  1.3139771 , -1.065803  , -0.00446671,  0.4457057 ,
        -0.9192658 ,  0.12247662,  1.2407341 , -0.01393995,  0.52465075,
        -0.11337585, -1.3417014 ]], dtype=float32)

In [10]:
W = model.get_layer('emb').get_weights()[0]
W.shape

(32545, 32)